In [1]:
from hydradx.model.processing import import_binance_prices
from hydradx.model.amm.global_state import GlobalState, OmnipoolState, historical_prices
from hydradx.model.amm.global_state import liquidate_against_omnipool_and_settle_otc

from hydradx.model.amm.agents import Agent
from hydradx.model.amm.trade_strategies import omnipool_arbitrage, back_and_forth, invest_all
from hydradx.model.amm.omnipool_amm import dynamicadd_lrna_fee, dynamicadd_asset_fee
from hydradx.model import run
from hydradx.model.amm.liquidations import money_market, CDP
from hydradx.model.amm.global_state import update_prices_and_process




In [2]:
price_list = import_binance_prices(['BTC'], start_date='April 30 2024', days=2)
print(price_list)
price_list_padded = []
for x in price_list:
    prices_padded = {'BTC': x['BTC'], 'ETH': 3000, 'DOT': 7, 'HDX': 0.015}
    price_list_padded.append(prices_padded)

[{'BTC': 63866.0}, {'BTC': 63882.27}, {'BTC': 63844.18}, {'BTC': 63806.0}, {'BTC': 63825.99}, {'BTC': 63824.28}, {'BTC': 63806.01}, {'BTC': 63802.01}, {'BTC': 63804.0}, {'BTC': 63804.99}, {'BTC': 63804.98}, {'BTC': 63800.0}, {'BTC': 63800.01}, {'BTC': 63810.0}, {'BTC': 63806.2}, {'BTC': 63809.99}, {'BTC': 63788.73}, {'BTC': 63796.41}, {'BTC': 63806.0}, {'BTC': 63802.0}, {'BTC': 63801.99}, {'BTC': 63783.33}, {'BTC': 63791.86}, {'BTC': 63788.01}, {'BTC': 63771.73}, {'BTC': 63764.31}, {'BTC': 63792.69}, {'BTC': 63776.77}, {'BTC': 63792.69}, {'BTC': 63792.68}, {'BTC': 63792.69}, {'BTC': 63789.82}, {'BTC': 63758.84}, {'BTC': 63733.99}, {'BTC': 63736.0}, {'BTC': 63744.0}, {'BTC': 63743.99}, {'BTC': 63731.11}, {'BTC': 63731.12}, {'BTC': 63722.57}, {'BTC': 63717.71}, {'BTC': 63711.91}, {'BTC': 63711.9}, {'BTC': 63717.72}, {'BTC': 63704.0}, {'BTC': 63688.05}, {'BTC': 63706.0}, {'BTC': 63696.81}, {'BTC': 63721.05}, {'BTC': 63726.97}, {'BTC': 63731.11}, {'BTC': 63755.99}, {'BTC': 63750.01}, {'BTC

In [3]:


assets = {
    'HDX': {'usd price': price_list_padded[0]['HDX'], 'weight': 0.05},
    'USDT': {'usd price': 1, 'weight': 0.20},
    'BTC': {'usd price': price_list_padded[0]['BTC'], 'weight': 0.10},
    'ETH': {'usd price': price_list_padded[0]['ETH'], 'weight': 0.10},
    'DOT': {'usd price': price_list_padded[0]['DOT'], 'weight': 0.40},
}

lrna_price_usd = 38
initial_omnipool_tvl = 25000000
liquidity = {}
lrna = {}

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd
    
omnipool = OmnipoolState(
        tokens={
            tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
        },
        # oracles={'fee_raise': 50},
        # lrna_fee=dynamicadd_lrna_fee(
        #     minimum=0.0005,
        #     amplification=1,
        #     raise_oracle_name='fee_raise',
        #     decay=0.00001,
        #     fee_max=0.1,
        # ),
        lrna_fee=0.0005,
        # asset_fee=dynamicadd_asset_fee(
        #     minimum=0.0025,
        #     amplification=1,
        #     raise_oracle_name='fee_raise',
        #     decay=0.00005,
        #     fee_max=0.4,
        # ),
        asset_fee=0.0025,
        preferred_stablecoin='USDT',
    )

borrowing_agent = Agent()
init_ltv = 0.69
cdp_init = CDP('USDT', 'BTC', 10000, 10000/price_list_padded[0]['BTC']/init_ltv)
mm = money_market(
    liquidity={"USDT": 1000000, "BTC": 1000},
    oracles={("BTC", "USDT"): price_list_padded[0]['BTC']},
    liquidation_threshold=0.7,
    cdps=[(borrowing_agent, cdp_init.copy())],
    min_ltv=0.6
)

# price_transform = historical_prices(price_list_padded)
# liquidate_transform = liquidate_against_omnipool_and_settle_otc('omnipool', 'Arbitrageur')
# def compound_transform(state: GlobalState) -> GlobalState:
#     return liquidate_transform(price_transform(state))

initial_state = GlobalState(
    pools={'omnipool': omnipool},
    agents = {
        'Arbitrageur': Agent(
            holdings={'HDX': 1000000000, 'USDT': 1000000000, 'DOT': 1000000000, 'LRNA': 1000000000, 'ETH': 1000000000, 'BTC': 1000000000},
            trade_strategy=omnipool_arbitrage(
                pool_id='omnipool',
                arb_precision=3,
                # skip_assets=['HDX']
            )
        ),
    },
    evolve_function=update_prices_and_process('omnipool', 'Arbitrageur', price_list_padded),
    external_market={tkn: assets[tkn]['usd price'] for tkn in assets},
    money_market=mm,
)
time_steps = len(price_list)-1
print(cdp_init)

CDP(USDT, BTC, 10000, 0.2269243983213041)


In [4]:
print(f'Running with {time_steps} time steps')
events = run.run(initial_state, time_steps=time_steps)
print(len(events))

Running with 14399 time steps
Starting simulation...
Execution time: 6.147 seconds.
14399


In [5]:
print(events[-1].money_market.cdps[0][1])

CDP(USDT, BTC, 0, 0.0664583840865671)


In [6]:
print(min([x['BTC'] for x in price_list]))
print(price_list[0]['BTC'])
min_ratio = min([x['BTC'] for x in price_list]) / price_list[0]['BTC']
print(min_ratio)

56594.01
63866.0
0.8861367550809508
